<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
<span><img src="media/both-logos-small-alpha.png" style="display:inline" /></span>
</div>

# Autres bibliothèques de visualisation

## Complément - niveau basique

Pour conclure cette séquence sur les outils de visualisation, nous allons très rapidement évoquer des alternatives à la bibliothèque `matplotlib`, sachant qu'il existe en réalité un très grand nombre de bibliothèques en développement dans ce domaine en pleine expansion.

### Le poids du passé

On a vu que `matplotlib` est un outil relativement complet. Toutefois, on peut lui reprocher deux défauts majeurs.

* D'une part, `matplotlib` a choisi d'offrir une interface aussi proche que possible de ce qui existait préalablement en MatLab. C'est un choix tout à fait judicieux dans l'optique d'attirer la communauté utilisatrice de MatLab à des outils open source basés sur Python et numpy. Mais en contrepartie cela implique d'adopter tels quels des choix de conception.

* Et notamment, en suivant cette approche on hérite d'un modèle mental qui est plus orienté vers la sortie vers du papier que vers la création de documents interactifs.

Ceci, ajouté à l'explosion du domaine de l'analyse et de la visualisation de données, explique la largeur de l'offre en matière de bibliothèques de visualisation alternatives.

### `bokeh`

Parmi celles-ci, nous voulons vous signaler notamment la bibliothèque [`bokeh`](https://bokeh.pydata.org/en/latest/), qui est développée principalement par Anaconda, dans un modèle open source.

`bokeh` présente quelques bonnes propriétés qui nous semblent mériter d'être signalées.

Pour commencer cette bibliothèque utilise une architecture qui permet de *penser la visualisation comme quelque chose d'interactif* (disons une page html), et non pas de figé comme lorsqu'on pense en termes de feuille de papier. Notamment elle permet de faire collaborer du code Python avec du code JavaScript, qui offre immédiatement des possibilités bien plus pertinentes lorsqu'il s'agit de créer des interactions utilisateur qui soient attractives et efficaces. Signalons en passant, à cet égard, qu'elle utilise [la librairie JavaScript `d3.js`](https://d3js.org/), qui est devenu un standard de fait plus ou moins incontournable dans le domaine de la visualisation.

En tout état de cause, elle offre une interface de programmation qui tient compte d'environnements comme les notebooks, ce qui peut s'avérer un atout précieux si vous utilisez massivement ce support, comme on va le voir, précisément, dans ce notebook.

Il peut aussi être intéressant de savoir que `bokeh` offre des possibilités natives de [visualisation de graphes](https://bokeh.pydata.org/en/latest/docs/user_guide/graph.html) et de [ données géographiques](https://bokeh.pydata.org/en/latest/docs/user_guide/geo.html#).

Par contre à ce stade du développement, la visualisation en 3D n'est sans doute pas le point fort de `bokeh`. C'est une option qui reste possible (voir [par exemple ceci](https://github.com/bokeh/bokeh/tree/master/examples/app/surface3d)), mais cela est pour l'instant considéré comme une extension de la librairie, et donc n'est accessible qu'au prix de l'écriture de code javascript.

Pour une présentation plus complète, je vous renvoie à [la documentation utilisateur](https://bokeh.pydata.org/en/latest/docs/user_guide.html).

### `bokeh` dans les notebooks

Nous allons rapidement illustrer ici comment `bokeh` s'interface avec l'environnement des notebooks pour créer une visualisation interactive. Vous remarquerez que dans le code qui suit, on n'a **pas eu besoin de mentionner** de *magic* ipython, comme lorsqu'on avait du faire dans le complément sur les notebooks interactifs :

```python
%matplotlib notebook
```

In [ ]:
import numpy as np

In [ ]:
# l'attirail de notebooks interactifs
from ipywidgets import interact, fixed, FloatSlider

In [ ]:
# les imports pour bokeh
from bokeh.plotting import figure, show
# dans la rubrique entrée-sortie, on trouve
# les outils pour produire du html
#  (le mode par défaut)
# ou pour interactig avec un notebook
from bokeh.io import push_notebook, output_notebook

In [ ]:
# c'est cette déclaration qui remplace
# si on veut la magic  '%matplotlib notebook'
output_notebook()

*****

In [ ]:
# on crée un objet figure
fig1 = figure(
    title="fonctions trigonométriques",
    plot_height=300, plot_width=600,
    # c'est là notamment qu'on précise
    # l'intervalle en y
    y_range=(-5, 5),
)

In [ ]:
# on initialise la figure en créant
# un objet courbe
x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)
courbe_trigo = fig1.line(x, y, color="#2222aa", line_width=3)

In [ ]:
# la fonction de mise à jour, qui sera connectée
# à interact
def update_trigo(function_name, frequence=1,
                 amplitude=1, phase=0,
                 # l'objet handle correspond
                 # à une figure à mettre à jour
                 *, handle):
    # juste une astuce pour pouvoir choisir
    # la fonction trigonométrique, qu'on recherche
    # dans le module numpy
    func = getattr(np, function_name)
    # c'est ici qu'on modifie les données
    # utilisées pour produire la courbe
    courbe_trigo.data_source.data['y'] = \
       amplitude * func(frequence * x + phase)
    # et c'est ici qu'on provoque la mise à jour
    push_notebook(handle)

In [ ]:
# au moment où matérialise l'objet figure
# on récupère une `handle` qui lui correspond
handle1 = show(fig1, notebook_handle=True)

In [ ]:
# maintenant on peut créer un interacteur
interact(update_trigo, function_name=["sin", "cos", "tan"],
         frequence=(1,20),
         amplitude=[0.5, 1, 3, 5],
         phase=(0, 2*np.pi, 0.05),
         handle=fixed(handle1),
        );

*****

### Exercice : distribution uniforme

Voyons un deuxième exemple avec `bokeh`. Vous pouvez prendre ceci comme un exercice, et le faire de votre côté avant de lire la suite du notebook.

On veut ici écrire un outil pour afficher une distribution de points dans une ellipse, de taille et de position réglables.

Dans la solution que vous trouverez ci-dessous, le nombre de points `N` dans la distribution est supposé constant; en fait, dans ce code on va tirer au sort une bonne fois pour toutes `N` points dans le cercle de rayon 1, avec une distribution uniforme, et simplement déformer cette distribution pour occuper l'espace cible.

On se donne donc comme réglages :

* `dx` et `dy`, les coordonnées du centre de l'ellipse,
* `rx` et `ry` les rayons en x et en y de l'ellipse,
* et enfin `alpha` l'angle de rotation de l'ellipse.

****

In [ ]:
# petit utilitaire pour calculer la distribution
# uniforme de départ
def uniform_distribution(N):
    # on tire au hasard un rho et un rayon
    rhos = 2 * np.pi * np.random.sample(N)
    rads = np.random.sample(N)
    # il faut prendre la racine carrée du rayon
    # sinon ce n'est pas uniforme dans le plan
    circle_x = np.sqrt(rads) * np.cos(rhos)
    circle_y = np.sqrt(rads) * np.sin(rhos)
    return circle_x, circle_y

##### Les grandeurs constantes

In [ ]:
# les grandeurs constantes
N = 1000

In [ ]:
# on calcule la distribution initiale
# (celle-ci est vraiment uniforme)
# dans le cercle de rayon 1
x0, y0 = uniform_distribution(N)

In [ ]:
# et aussi:
# pour que ce soit plus joli je tire au hasard
# des couleurs, et des rayons pour les points

# le rouge entre 50 et 250
reds = 50 + 200 * np.random.random(size=N)
# le vert entre 30 et 250
greens = 30 + 220 * np.random.random(size=N)
# la mise en forme des couleurs
# le bleu est constant à 150
colors = [
    f"#{int(red):02x}{int(green):02x}{150:02x}"
    for red, green in zip(reds, greens)
]

# les rayons des points; entre 0.05 et 0.25
radii = 0.05 + np.random.random(size=N) * .20

##### Création de la figure initiale

In [ ]:
# c'est ici qu'on commence à faire du bokeh
# les choix des bornes sont très arbitraires
fig2 = figure(
    title="distribution pseudo-uniforme",
    plot_height=250, plot_width=250,
    x_range=(-10, 10),
    y_range=(-10, 10),
)

In [ ]:
# on crée le nuage de points dans la figure
cloud = fig2.circle(
    x0, y0,
    radius = radii,
    fill_color=colors, fill_alpha=0.6,
    line_color=None, line_width=.1
)

##### Mise à jour de la figure

In [ ]:
# c'est cette fonction qu'on passe à interact
def update_cloud(rx, ry, dx, dy, alpha, handle):
    # on recalcule les x et y
    # à partir des valeurs initiales
    s, c = np.sin(alpha), np.cos(alpha)
    x = dx + c * rx * x0 - s * ry * y0
    y = dy + s * rx * x0 + c * ry * y0
    cloud.data_source.data['x'] = x
    cloud.data_source.data['y'] = y
    push_notebook(handle)

##### Il n'y a plus qu'à ...

In [ ]:
handle2 = show(
    fig2,
    notebook_handle=True)

In [ ]:
interact(
  update_cloud,
  rx=FloatSlider(min=.5, max=8,
                 step=.1, value=1.),
  ry=FloatSlider(min=.5, max=8,
                 step=.1, value=1.),
  dx=(-3, +3, .2),
  dy=(-3, +3, .2),
  alpha=FloatSlider(
    min=0., max=np.pi,
    step=.05, value=0.),
  handle=fixed(handle2)
);

### Autres bibliothèques

Pour terminer cette digression sur les solutions alternatives à `matplotlib`, j'aimerais vous signaler enfin rapidement [la bibliothèque `plotly`](https://plot.ly/).

Cette bibliothèque est disponible en open source, et l'offre commerciale de plotly est tournée vers le conseil autour de cette technologie. Comme pour `bokeh`, elle est conçue comme un hybride entre Python et JavaScript, au dessus de `d3.js`. En réalité, elle présente même la particularité d'offrir une API unique disponible depuis Python, JavaScript, et R.

Comme on l'a dit en introduction, l'offre dans ce domaine est pléthorique, aussi si vous avez un témoignage à apporter sur une expérience que vous avez eue dans ce domaine, nous serons ravis de vous voir la partager dans le forum du cours.